# Auxiliar Table 7

In [ ]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.ThreeCatAnalysis as thr
from luxgiant_clinical.Helpers import categories_recoder

import pandas as pd
import numpy as np

In [ ]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [ ]:
# filter data to keep only patients with PD subtype information

mask_patients= (df['Status']=='Patient')
mask_subtype = ~df['subtype'].isnull()
mask_off     = (df['on_off']=='OFF: Off is the typical functional state when patien ts have a poor response in spite of taking medications.')

df_cases = df[mask_patients & mask_off & mask_subtype].reset_index(drop=True)
del df

df_cases.shape

## Summary statistics for demography and lifestyle

In [ ]:
# rename and select first group of variables
variables_dict_1 = {
    "age_at_onset"              : "Age at Onset (years)",
    "agecat_1"                  : "Onset <50 years",
    "agecat_2"                  : "Onset <40 years",
    "age"                       : "Age at Recruitment (years)",
    "PD_duration"               : "Duration of Disease (years)",
    "sex"                       : "Male",
    "bmi_comp"                  : "Body Mass Index",
    "years_of_education"        : "Years of Education",
    "work"                      : "Currently Employed",
    "nature_of_work___1"        : "Agriculture as occupation",
    "over_your_lifetime_have_yo": "Life time direct exposure to pesticide/insecticide/fungicide",
    "have_you_ever_had_a_head_i": "Head injury or concussion",
    "in_your_lifetime_have_you" : "Smoked 100 or more cigarettes in lifetime",
    "in_your_lifetime_have"     : "Regular consumption of caffeinated drinks for >6months",                                
}
variables_1 = ['participant_id', 'subtype'] + list(variables_dict_1.keys())

# statistical measures
stats_meas = {
    'mean'  : ["Age at Recruitment (years)", "Age at Onset (years)", "Body Mass Index"],
    'n'     : ["Onset <50 years", "Onset <40 years", "Male", "Currently Employed", 
               "Agriculture as occupation", 
               "Life time direct exposure to pesticide/insecticide/fungicide", 
               "Smoked 100 or more cigarettes in lifetime", 
               "Regular consumption of caffeinated drinks for >6months", 
               "Head injury or concussion"],
    'median': ["Duration of Disease (years)", "Years of Education"]
}

groups = ['Tremor Dominant', 'Indeterminate', 'Postural instability and gait difficulty']

In [ ]:
# keep only variables for analysis

df_cases_1 = df_cases[variables_1].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict_1)

In [ ]:
# recode variable values

recode_dict = {
    "Currently Employed"                :{'Yes':1, 'No':0},
    "Agriculture as occupation"         :{'Checked':1, 'Unchecked':0},
    "Male"                              :{'Male':1, 'Female':0},
    "Onset <50 years"                   :{'<50':1, '>=50':0},
    "Onset <40 years"                   :{'<40':1, '>=40':0}
}
for key in recode_dict.keys():
    df_cases_1 = categories_recoder(df_cases_1, [key], recode_dict[key])

recode = ["Life time direct exposure to pesticide/insecticide/fungicide", 
          "Smoked 100 or more cigarettes in lifetime",
          "Regular consumption of caffeinated drinks for >6months", "Head injury or concussion"]
mapping = {'Yes':1, 'No':0, 'Dont Know':np.nan, 'Refused':np.nan, 'Possibly':np.nan}

df_cases_1 = categories_recoder(df_cases_1, recode, mapping)


In [ ]:
summary1 = [
    thr.report_mean_std(
        data_df    =df_cases_1,
        variables  =stats_meas['mean'],
        groups     =groups,
        grouping_by='subtype'
    ),
    thr.report_median_iqr(
        data_df    =df_cases_1,
        variables  =stats_meas['median'],
        groups     =groups,
        grouping_by='subtype'
    ),
    thr.report_proportion(
        data_df    =df_cases_1,
        variables  =stats_meas['n'],
        groups     =groups,
        grouping_by='subtype'
    )
]

In [ ]:
# rename and select second group of variables

variables_dict_2 = {
    "medical_history_metabolic___1" : "Diabetes",
    "medical_history_metabolic___3" : "High Cholesterol",
    "medical_history_cardiovasc___2": "Hypertension",
    "medical_history_cardiovasc___3": "History of CAD",
    "medical_history_neurologic___7": "Stroke",
    "medical_history_psychologi___3": "History of Depression",
    "medical_history_psychologi___1": "History of Anxiety",
    "medical_history_cancer___1"    : "Melanoma" ,
    "medical_history_cancer___2"    : "Prostate Cancer" ,
    "medical_history_cancer___3"    : "Other Cancers",
    "family_member_diagnosed_wi"    : "Family History of PD",
    "family_member_diagnosed_wi_2"  : "Family History of Tremor"
}
variables_2 = ['participant_id', 'subtype'] + list(variables_dict_2.keys())

# statistical measures
stats_meas2 = {
    'n'     : ["Stroke", "Melanoma", "Prostate Cancer", "Other Cancers", "Diabetes", "High Cholesterol", "Hypertension",
               "History of CAD", "History of Anxiety", "History of Depression", "Family History of PD", 
               "Family History of Tremor"],
}

In [ ]:
# keep only variables for analysis

df_cases_2 = df_cases[variables_2].copy()
df_cases_2 = df_cases_2.rename(columns=variables_dict_2)

In [ ]:
# recode variable values
fam_hist = ["Family History of PD", "Family History of Tremor"]
remaining= [feat for feat in stats_meas2['n'] if feat not in fam_hist]

df_cases_2 = categories_recoder(df_cases_2, fam_hist, {"Yes":1, "No":0})
df_cases_2 = categories_recoder(df_cases_2, remaining, {"Checked":1, "Unchecked":0})

In [ ]:
summary2 = [
    thr.report_proportion(
        data_df    =df_cases_2,
        variables  =stats_meas2['n'],
        groups     =groups,
        grouping_by='subtype',
        subheader  ='Medical History'
    ),
]

In [ ]:
# concatenate all results

df_7 = pd.concat(
    summary1 + summary2, axis=0,ignore_index=True
)

In [ ]:
df_cases_3 = df_cases_1[~(df_cases_1['subtype']=='Indeterminate')].reset_index(drop=True)
df_cases_4 = df_cases_2[~(df_cases_2['subtype']=='Indeterminate')].reset_index(drop=True)

In [ ]:
summary3 = [
    thr.bonferroni_mean_std(
        data_df      =df_cases_3, 
        variables    =stats_meas['mean'], 
        groups       =['Tremor Dominant',  'Postural instability and gait difficulty'],
        grouping_by  ='subtype',
        correc_factor=3
    ),
    thr.bonferroni_median_iqr(
        data_df      =df_cases_3, 
        variables    =stats_meas['median'], 
        groups       =['Tremor Dominant',  'Postural instability and gait difficulty'],
        grouping_by  ='subtype',
        correc_factor=3
    ),
    thr.bonferroni_proportions(
        data_df      =df_cases_3, 
        variables    =stats_meas['n'], 
        groups       =['Tremor Dominant',  'Postural instability and gait difficulty'],
        grouping_by  ='subtype',
        correc_factor=3
    )
]

In [ ]:
summary4 = [
    thr.bonferroni_proportions(
        data_df      =df_cases_4, 
        variables    =stats_meas2['n'], 
        groups       =['Tremor Dominant',  'Postural instability and gait difficulty'],
        grouping_by  ='subtype',
        correc_factor=3
    ),
]

In [ ]:
# concatenate all results

df_71 = pd.concat(
    summary3 + summary4, axis=0,ignore_index=True
)

In [ ]:
df_7 = thr.final_formatter(
    overall_df =df_7, 
    adjusted_df=[df_71], 
    groups     =['Tremor Dominant', 'Indeterminate', 'Postural instability and gait difficulty']
)

In [ ]:
row_order = [
    "Age at Onset (years)", "Onset <50 years", "Onset <40 years", "Age at Recruitment (years)",
    "Duration of Disease (years)", "Male", "Body Mass Index", "Years of Education", "Currently Employed",
    "Family History of PD", "Family History of Tremor", "Agriculture as a occupation",
    "Life time direct exposure to pesticide/insecticide/fungicide", "Head injury or concussion",
    "Smoked 100 or more cigarettes in lifetime", 
    "Regular consumption of caffeinated drinks for >6months", "Diabetes", "High Cholesterol",
    "Hypertension", "History of CAD", "Stroke", "History of Depression", "History of Anxiety",
    "Melanoma", "Prostate Cancer" , "Other Cancers"
]

In [ ]:
df_final = pd.DataFrame(columns=df_7.columns)

for row in row_order:
    df_final = pd.concat(
        [df_final, df_7[df_7['Variable']==row]], axis=0, ignore_index=True
    )

df_final

In [ ]:
df_final.to_csv(os.path.join(path_to_lib, 'data/auxiliar/aux_table_7.csv'), index=False)